In [27]:
import numpy as np
import pandas as pd
from numpy.random import uniform, normal  
import math
from math import pi
from scipy.optimize import Bounds
from scipy.optimize import minimize

In [28]:
def J_square(sigma_square,tau_square):
    return tau_square/(tau_square+sigma_square)

In [29]:
# k: the number of studies
k_list = [10,30,50,100]
# each element of `parameter_constellation` has the form (mu, sigma^2,tau^2) 
# where tau^2 is the between-study variance, sigma^2*u_i is the within-study variance
parameter_constellation = [(0,12,4),(0,9,4),(0,4,4),(0,2,6)]
# heterogeniety measure: J^2
J_square_list = list()
for element in parameter_constellation:
    sigma_square_i = element[1]
    tau_square_i = element[2]
    J_square_list.append(J_square(sigma_square_i,tau_square_i))
print('J square =',J_square_list)

J square = [0.25, 0.3076923076923077, 0.5, 0.75]


## method1: trying to minimize the negative log likelihood

In [20]:
# define the negative log likelihood function
# minimize negative log likelihood function -> maximize log likelihood function
def nll(x, D, u):
    # x = np.array([mu, sigma, tau])
    mu, sigma, tau = x[0], x[1], x[2]
    sigma_square, tau_square = sigma**2, tau**2
    return 1/2*np.sum((D - mu)**2/(tau_square + sigma_square*u) \
            +np.log(tau_square + sigma_square*u)+np.log(2*pi))

# Jacobian: the matrix of all its first-order partial derivatives
def jac(x, D, u):
    mu, sigma, tau = x[0], x[1], x[2]
    sigma_square, tau_square = sigma**2, tau**2
    a = D - mu
    b = tau_square + sigma_square * u
    # convert the derivative with respect to sigma/tau, instead of sigma^2/tau^2
    d_mu = np.sum(a / b)
    d_sigma = (np.sum((a**2 * u) / b**2) - np.sum(u / b)) * (2*sigma) 
    d_tau = (np.sum(a**2 / b**2) - np.sum(1./ b)) * (2*tau)
    return - np.array([d_mu, d_sigma, d_tau])

def func(x, D, u):
    return nll(x, D, u), jac(x, D, u)

#############

def simulation(k, mu,sigma,tau,num_replications=10000):
    # store the results of convergent mu/sigma/tau for each replication
    all_mu_ast, all_sigma_square_ast, all_tau_square_ast = [], [], []

    for i in range(num_replications):
        # generate k(the number of studies) D_i
        u_array = uniform(0.02,0.2,k)
        u_inv_array = 1 / u_array
        # x_i ~ N(mu,tau^2)
        x_array = normal(mu,tau,k)
        # D_i ~ N(x_i, sigma^2*u_i)
        D_array = normal(x_array,sigma*u_array**0.5,k)

        # Note that we cannot use the starting values suggested in Sangnawakij's paper
        # since given tau^2, sigma^2 and mu are calculated under the condition that the derivatives are zero
        # which leads to a stationary point

        # random but resonable initial point
        mu_0 = uniform(-2.0, 2.0)
        sigma_0 = uniform(0., 15.)
        tau_0 = uniform(0., 8.)
        x0 = np.array([mu_0, sigma_0, tau_0])
        minimizer = minimize(nll, x0, args=(D_array, u_array), method="BFGS", jac=jac, tol=1e-10)   
        #minimizer_kwargs = {"method": "BFGS", "args": (D_array, u_array), "jac":True}
        # corresponding mu/sigma/tau when the negative log likelihood function is minimized
        mu_ast, sigma_ast, tau_ast = minimizer.x
        sigma_square_ast = sigma_ast**2
        tau_square_ast = tau_ast**2
        # print("mu: {0}, sigma_square: {1}, tau_square: {2}".format(mu_ast, sigma_square_ast, tau_square_ast))
        all_mu_ast.append(mu_ast)
        all_sigma_square_ast.append(sigma_square_ast)
        all_tau_square_ast.append(tau_square_ast)

    # convert the list to numpy array
    all_mu_ast = np.array(all_mu_ast)
    all_sigma_square_ast = np.array(all_sigma_square_ast)
    all_tau_square_ast = np.array(all_tau_square_ast)
    
    # return the result for mu/sigma/tau in the form of: [bias, std]
    mu_info = [np.mean(all_mu_ast), np.mean(all_mu_ast)-mu, np.std(all_mu_ast)]
    sigma_square_info = [np.mean(all_sigma_square_ast),np.mean(all_sigma_square_ast)-sigma_square, np.std(all_sigma_square_ast)]
    tau_square_info = [np.mean(all_tau_square_ast),np.mean(all_tau_square_ast)-tau_square, np.std(all_tau_square_ast)]
    return mu_info, sigma_square_info, tau_square_info


In [22]:
%%time
#(mu, sigma^2,tau^2): (0,12,4),(0,9,4),(0,4,4),(0,2,6)
mu_mean, mu_bias, mu_se = [],[],[]
sigma_square_mean, sigma_square_bias, sigma_square_se = [],[],[]
tau_square_mean, tau_square_bias, tau_square_se = [],[],[]

for para_combination in parameter_constellation:
    for k in k_list:
        # get the parameters
        mu = para_combination[0]
        sigma_square = para_combination[1]
        sigma = np.sqrt(sigma_square)
        tau_square = para_combination[2]
        tau = np.sqrt(tau_square)
        # simulate
        each_simulation = simulation(k, mu,sigma,tau,num_replications=10000)
        # append the results to lists
        mu_mean.append(each_simulation[0][0])
        mu_bias.append(each_simulation[0][1])
        mu_se.append(each_simulation[0][2])
        
        sigma_square_mean.append(each_simulation[1][0])
        sigma_square_bias.append(each_simulation[1][1])
        sigma_square_se.append(each_simulation[1][2]) 
        
        tau_square_mean.append(each_simulation[2][0])
        tau_square_bias.append(each_simulation[2][1])
        tau_square_se.append(each_simulation[2][2])

#print('mean of mu_hat:', mu_mean)
#print('bias of mu_hat:', mu_bias)
#print('standard error of mu_hat:', mu_se)
#print('----------------------------------------------------------------')
#print('mean of sigma_square_hat:', sigma_square_mean)
#print('bias of sigma_square_hat:', sigma_square_bias)
#print('standard error of sigma_square_hat:', sigma_square_se)
#print('----------------------------------------------------------------')
#print('mean of tau_square_hat:', tau_square_mean)
#print('bias of tau_square_hat:', tau_square_bias)
#print('standard error of tau_square_hat:', tau_square_se)


CPU times: user 9min 34s, sys: 1.21 s, total: 9min 35s
Wall time: 9min 36s


In [26]:
# from pd.dataframe to latex table
df = pd.DataFrame({'J_square': [i for i in J_square_list for _ in range(4)],
                   'k': k_list*4,
                   #'mean_mu': mu_mean,
                   #'mean(sigma^2)': sigma_square_mean,
                   #'mean(tau^2)': tau_square_mean,
                   'bias(mu)': mu_bias,
                   'bias(sigma^2)': sigma_square_bias,
                   'bias(tau^2)': tau_square_bias,
                   'standard_error(mu)': mu_se,
                   'standard_error(sigma^2)': sigma_square_se,
                   'standard_error(tau^2)': tau_square_se})
print(df.to_latex(index=False))

\begin{tabular}{rrrrrrrr}
\toprule
 J\_square &    k &  bias(mu) &  bias(sigma\textasciicircum 2) &  bias(tau\textasciicircum 2) &  standard\_error(mu) &  standard\_error(sigma\textasciicircum 2) &  standard\_error(tau\textasciicircum 2) \\
\midrule
 0.250000 &   10 &  0.007375 &       8.516906 &    -1.435006 &            0.767052 &                24.644566 &               5.834693 \\
 0.250000 &   30 & -0.002435 &       5.767365 &    -0.814487 &            0.431672 &                19.331316 &               2.058517 \\
 0.250000 &   50 &  0.000513 &       3.604686 &    -0.491619 &            0.332171 &                16.491241 &               1.788568 \\
 0.250000 &  100 &  0.005041 &       1.818044 &    -0.253294 &            0.235613 &                12.573685 &               1.381885 \\
 0.307692 &   10 &  0.003879 &       9.559981 &    -1.535135 &            0.722793 &                22.255735 &               2.860925 \\
 0.307692 &   30 &  0.000297 &       6.341221 &    -0.820384

## method2: trying to calculate the maximum log likehood estimates of mu/sigma/tau iteratively (fixed point algorithm in Sangnawakij's paper)

### starting value
$$\tau_0^2 = 0$$
$$\hat{\mu_0} = \frac{\sum \limits_{i=1}^{k} \frac{D_i}{u_i}}{\sum \limits_{i=1}^{k} \frac{1}{u_i}}$$
$$\hat{\sigma_0}^2 = \frac{1}{k} \sum \limits_{i=1}^{k} \frac{(D_i - \hat{\mu_0})^2}{u_i}$$

### iterative steps
$$\mu_{s+1} = \frac{\sum \limits_{i=1}^{k}\frac{D_i}{\tau_s^2+\sigma_s^2 u_i}}{\sum \limits_{i=1}^{k} \frac{1}{\tau_s^2+\sigma_s^2 u_i}}$$
$$\sigma_{s+1}^2 = \frac{\sum \limits_{i=1}^{k} \frac{(D_i - \mu_s)^2 u_i - \tau_s^2 u_i}{(\tau_s^2 + \sigma_s^2 u_i)^2}}{\sum \limits_{i=1}^{k}\frac{u_i^2}{(\tau_s^2+\sigma_s^2 u_i)^2}} $$
$$\tau_{s+1}^2 = \frac{\sum \limits_{i=1}^{k} \frac{(D_i - \mu_s)^2 - \sigma_s^2 u_i}{(\tau_s^2 + \sigma_s^2 u_i)^2}}{\sum \limits_{i=1}^{k}\frac{1}{(\tau_s^2+\sigma_s^2 u_i)^2}} $$

In [9]:
def simulation_fp(k, actual_mu,actual_sigma,actual_tau,num_replications=10000,stopping_criteria = 1e-10):
    actual_sigma_square = actual_sigma**2
    actual_tau_square = actual_tau**2
    
    count_array = np.array([])
    mu_array = np.array([])
    tau_square_array = np.array([])
    sigma_square_array = np.array([])

    #for num in range(replications):
    while len(count_array) < num_replications:
        # firstly, generate all u_i and D_i from normal distribution
        u_array = uniform(0.02,0.2,k)
        # xi_i ~ N(mu,tau^2)
        xi_array = normal(actual_mu,actual_tau,k)
        # D_i ~ N(x_i, sigma^2*u_i)
        D_array = normal(xi_array,actual_sigma*u_array**0.5,k)
        u_inv_array = 1/u_array

        # starting value
        tau_square_hat_0 = 0
        mu_hat_0 = np.sum(D_array*u_inv_array)/np.sum(u_inv_array)
        sigma_square_hat_0 = 1/k*np.sum((D_array - mu_hat_0)**2/u_array) 

        log_likelihood = -1/2*np.sum((D_array - mu_hat_0)**2/(tau_square_hat_0 + sigma_square_hat_0*u_array)+np.log(tau_square_hat_0 + sigma_square_hat_0*u_array)+np.log(2*pi))
        #print('The log likelihood corresponding to starting values is', log_likelihood)

        mu = mu_hat_0
        tau = np.sqrt(tau_square_hat_0)
        sigma= np.sqrt(sigma_square_hat_0)



        #########
        count = 0
        error = 1
        while error > stopping_criteria:
            tau_square = tau**2
            sigma_square = sigma**2

            # calculate the common terms firstly to avoid repeated calculation
            common_term = 1/(tau_square + sigma_square*u_array)
            common_term_square = common_term**2
            D_minus_u_square = (D_array - mu)**2

            # iterative equations
            mu_updated = np.sum(D_array*common_term) / np.sum(common_term)
            sigma_square_updated = np.sum((D_minus_u_square*u_array - tau_square*u_array)*common_term_square)/np.sum(u_array**2*common_term_square)
            tau_square_updated = np.sum((D_minus_u_square - sigma_square*u_array)*common_term_square)/np.sum(common_term_square)
            log_likelihood = -1/2*np.sum((D_array - mu_updated)**2/(tau_square_updated + sigma_square_updated*u_array)+np.log(tau_square_updated + sigma_square_updated*u_array)+np.log(2*pi))


            # use sigma/tau -> unconstrained problem
            sigma_updated = np.sqrt(sigma_square_updated)
            tau_updated = np.sqrt(tau_square_updated)

            #print('log likelihood:',log_likelihood)

            #print(abs(mu_updated-mu))
            #print(abs(sigma_updated-sigma))
            #print(abs(tau_updated-tau))
            #print('----------------------------------')

            error = max(abs(mu_updated-mu),abs(sigma_updated-sigma),abs(tau_updated-tau))
            #print('max error = ',error)

            mu = mu_updated
            sigma = sigma_updated
            tau = tau_updated
            count += 1

        # only keep the cases which is convergent and sigma_square/tau_square are non-negative
        if (count>1) and (math.isnan(mu) == False) and (math.isnan(sigma) == False) and (math.isnan(tau) == False):
            count_array = np.append(count_array, count)
            mu_array = np.append(mu_array, mu)
            sigma_square_array = np.append(sigma_square_array,sigma_square)
            tau_square_array = np.append(tau_square_array, tau_square)
            
            
    mu_info = [np.mean(mu_array), np.mean(mu_array)-actual_mu, np.std(mu_array)]
    sigma_square_info = [np.mean(sigma_square_array),np.mean(sigma_square_array)-actual_sigma_square, np.std(sigma_square_array)]
    tau_square_info = [np.mean(tau_square_array),np.mean(tau_square_array)-actual_tau_square, np.std(tau_square_array)]
    return mu_info, sigma_square_info, tau_square_info
    


In [14]:
%%time

#(mu, sigma^2,tau^2): (0,12,4),(0,9,4),(0,4,4),(0,2,6)
mu_mean_fp, mu_bias_fp, mu_se_fp = [],[],[]
sigma_square_mean_fp, sigma_square_bias_fp, sigma_square_se_fp = [],[],[]
tau_square_mean_fp, tau_square_bias_fp, tau_square_se_fp = [],[],[]

for para_combination in parameter_constellation:
    for k in k_list:
        # get the parameters
        actual_mu = para_combination[0]
        actual_sigma_square = para_combination[1]
        actual_sigma = np.sqrt(actual_sigma_square)
        actual_tau_square = para_combination[2]
        actual_tau = np.sqrt(actual_tau_square)
        # simulate
        each_simulation_fp = simulation_fp(k, actual_mu,actual_sigma,actual_tau,num_replications=10000,stopping_criteria = 1e-10)
        # append the results to lists
        mu_mean_fp.append(each_simulation_fp[0][0])
        mu_bias_fp.append(each_simulation_fp[0][1])
        mu_se_fp.append(each_simulation_fp[0][2])
        
        sigma_square_mean_fp.append(each_simulation_fp[1][0])
        sigma_square_bias_fp.append(each_simulation_fp[1][1])
        sigma_square_se_fp.append(each_simulation_fp[1][2]) 
        
        tau_square_mean_fp.append(each_simulation_fp[2][0])
        tau_square_bias_fp.append(each_simulation_fp[2][1])
        tau_square_se_fp.append(each_simulation_fp[2][2])



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in sqrt
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in sqrt
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in log


CPU times: user 47min 34s, sys: 5.41 s, total: 47min 40s
Wall time: 47min 45s


In [19]:
#print('mean of mu_hat:', mu_mean_fp)
#print('bias of mu_hat:', mu_bias_fp)
#print('standard error of mu_hat:', mu_se_fp)
#print('----------------------------------------------------------------')
#print('mean of sigma_square_hat:', sigma_square_mean_fp)
#print('bias of sigma_square_hat:', sigma_square_bias_fp)
#print('standard error of sigma_square_hat:', sigma_square_se_fp)
#print('----------------------------------------------------------------')
#print('mean of tau_square_hat:', tau_square_mean_fp)
#print('bias of tau_square_hat:', tau_square_bias_fp)
#print('standard error of tau_square_hat:', tau_square_se_fp)

# from pd.dataframe to latex table
df_fp = pd.DataFrame({'J_square': [i for i in J_square_list for _ in range(4)],
                   'k': k_list*4,
                   #'mean_mu': mu_mean,
                   #'mean(sigma^2)': sigma_square_mean,
                   #'mean(tau^2)': tau_square_mean,
                   'bias(mu)': mu_bias_fp,
                   'bias(sigma^2)': sigma_square_bias_fp,
                   'bias(tau^2)': tau_square_bias_fp,
                   'standard_error(mu)': mu_se_fp,
                   'standard_error(sigma^2)': sigma_square_se_fp,
                   'standard_error(tau^2)': tau_square_se_fp})
print(df_fp.to_latex(index=False))

\begin{tabular}{rrrrrrrr}
\toprule
 J\_square &    k &  bias(mu) &  bias(sigma\textasciicircum 2) &  bias(tau\textasciicircum 2) &  standard\_error(mu) &  standard\_error(sigma\textasciicircum 2) &  standard\_error(tau\textasciicircum 2) \\
\midrule
 0.250000 &   10 & -0.001991 &      11.291183 &    -1.656066 &            0.733642 &                18.556028 &               1.826939 \\
 0.250000 &   30 &  0.000815 &      10.635056 &    -1.265305 &            0.427278 &                14.990208 &               1.577687 \\
 0.250000 &   50 & -0.002636 &       8.665148 &    -1.002207 &            0.330845 &                13.728933 &               1.482482 \\
 0.250000 &  100 & -0.001472 &       5.402168 &    -0.633542 &            0.231044 &                11.059670 &               1.223603 \\
 0.307692 &   10 & -0.012355 &      12.725499 &    -1.755441 &            0.715221 &                17.255368 &               1.738297 \\
 0.307692 &   30 &  0.001083 &      11.742043 &    -1.375706

# This is method 1

If we let $c=\sqrt{a+b}$ then we'll find truth, but
$$
F(x) = \int_{-\infty}^x f(t) dt
$$
then we really know what's up, or
\begin{eqnarray}
a&=&b\\
&=&c
\end{eqnarray}
